In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
14,Зарубежные фильмы,04 Июн 23,"Рождение легенд / Spinning Gold (2023) BDRip 1080p от New-Team | Jaskier, TVShows",13.27 GB,11,45
17,Зарубежные фильмы,04 Июн 23,Питер Пэн и Венди / Peter Pan & Wendy (2023) WEB-DLRip-AVC от DoMiNo & селезень | D | Red Head Sound,1.72 GB,46,27
18,Зарубежные фильмы,03 Июн 23,Больше чем любовь (2023) WEBRip 1080p,3.13 GB,60,35
19,Зарубежные фильмы,03 Июн 23,Больше чем любовь (2023) WEBRip,1.44 GB,54,29
22,Зарубежные фильмы,03 Июн 23,"Питер Пэн и Венди / Peter Pan & Wendy (2023) WEB-DL-HEVC 2160p | 4K | HDR | Dolby Vision Profile 8 | D, P",11.14 GB,40,13
23,Зарубежные фильмы,03 Июн 23,Бессмертная гвардия / The Old Guard (2020) WEB-DLRip-AVC от DoMiNo & селезень | D,2.60 GB,51,12
24,Зарубежные фильмы,03 Июн 23,Мама мафия / Mafia Mamma (2023) BDRip | L,1.45 GB,260,104
27,Зарубежные фильмы,03 Июн 23,"Внутри / Inside (2023) BDRip 1080p от DoMiNo & селезень | P, A",8.95 GB,112,58
30,Зарубежные фильмы,03 Июн 23,Внутри / Inside (2023) HDRip от New-Team | Jaskier,1.46 GB,115,18
32,Зарубежные фильмы,03 Июн 23,Путь справедливости / Trail of Justice (2023) WEB-DLRip от ELEKTRI4KA | P,1.47 GB,161,64


In [8]:
id = [27]
my_df.loc[id, 'href']

27    http://rutor.info/torrent/918700/vnutri_inside-2023-bdrip-1080p-ot-domino-selezen-p-a
Name: href, dtype: object